In [20]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt
from top2vec import Top2Vec
from nltk.corpus import stopwords
from gensim.models import TfidfModel
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import load_iris

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
import ipynb
from ipynb.fs.full.topic_models import split_data, lda_vecs

In [25]:
training_data, test_data = split_data()

In [26]:
test_data.target.value_counts()

0    1499
1     501
Name: target, dtype: int64

In [27]:
################################################### lda ##############################################################################

In [28]:
lda_train_vecs = lda_vecs

In [29]:
lda_model = gensim.models.ldamodel.LdaModel.load("models/lda_train.model")

In [30]:
test_docs = test_data.allegation_desc

def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

lemmatized_texts = lemmatization(test_docs)

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

test_corpus = [id2word.doc2bow(text) for text in texts]

tdidf = TfidfModel(test_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(test_corpus)):
    bow = test_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    test_corpus[i] = new_bow

def get_test_vecs():
    lda_test_vecs = []
    for i in range(len(test_docs)):
        top_topics = lda_model.get_document_topics(test_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(10)]
        topic_vec.extend([len(test_docs.iloc[i])]) # length review
        lda_test_vecs.append(topic_vec)
    return lda_test_vecs

In [31]:
lda_test_vecs = get_test_vecs()

In [32]:
################################################### top2vec ##############################################################################

In [33]:
top_2_vec_model_train = Top2Vec.load("models/noso")
top_train_vecs = top_2_vec_model_train.document_vectors

In [34]:
def convert_test_data_to_list(data):
    test_docs = [x for x in data["allegation_desc"]]
    return test_docs

In [35]:
test_docs = convert_test_data_to_list(test_data)

top_2_vec_model_test = Top2Vec(test_docs, embedding_model_path="models/noso")

top_test_vecs = top_2_vec_model_test.document_vectors

2022-11-30 13:15:52,494 - top2vec - INFO - Pre-processing documents for training
2022-11-30 13:15:52,655 - top2vec - INFO - Creating joint document/word embedding
2022-11-30 13:15:57,575 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-30 13:16:10,128 - top2vec - INFO - Finding dense areas of documents
2022-11-30 13:16:10,285 - top2vec - INFO - Finding topics


In [36]:
############################################################# regress #################################################################

In [37]:
X_test = np.array(top_test_vecs)
y_test = np.array(test_data.target)

In [38]:
X = np.array(top_train_vecs)
y = np.array(training_data.target)

In [39]:
kf = KFold(5, shuffle=True, random_state=48)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True,
    ).fit(X_train_scale, y_train)

    test_data["lr_scores"] = lr.predict(scaler.transform(X_test))
    test_data[["lr_scores_prob_1"]] = lr.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = lr.predict(scaler.transform(X_val_scale))
    cv_lr_f1.append(f1_score(y_val, y_pred, average='weighted'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_scores"] = sgd.predict(scaler.transform(X_test))
    test_data[["sgd_scores_prob_1"]] = sgd.predict_proba(scaler.transform(X_test))[:, 1]

    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)

    test_data["sgd_huber_scores"] = sgd_huber.predict(scaler.transform(X_test))
    test_data[["sgd_huber_scores_prob_1"]] = sgd_huber.predict_proba(scaler.transform(X_test))[:, 1]
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average="weighted"))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning:

The line search algorithm did not converge

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning:

The line search algorithm did not converge

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning:

The line search algorithm did not converge

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning:

The line search algorithm did not converge

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning:

The line search algorithm did not converge



Logistic Regression Val f1: 0.535 +- 0.017
Logisitic Regression SGD Val f1: 0.533 +- 0.022
SVM Huber Val f1: 0.641 +- 0.010


In [40]:
test_pred = test_data[~((test_data.target == "0"))]
test_pred

,allegation_desc,meta,label,target,lr_scores,lr_scores_prob_1,sgd_scores,sgd_scores_prob_1,sgd_huber_scores,sgd_huber_scores_prob_1
1078,officer worked a paid detail in plain clothes ...,2014-0105-d,nan,1,0,0.098106,0,1.023363e-12,0,0.264397
1020,complaint was via webmail. police were called ...,2014-0001-n,nan,1,0,0.001504,0,4.948917e-17,0,0.354349
1374,officer accidentally left her service weapon u...,2014-0565-r,nan,1,0,0.000729,0,1.976547e-26,0,0.273947
1177,the complainant stated she was leaving her res...,2014-0250-c,nan,1,0,0.000971,0,1.090328e-16,0,0.350669
1523,complainant stated he was assaulted by three u...,2014-0826-c,nan,1,0,0.004884,0,3.529409e-12,0,0.345093
...,...,...,...,...,...,...,...,...,...,...
697,accused of refusing work assignment. employee ...,H-017-20,nan,1,0,0.453684,0,9.243473e-15,0,0.284905
1058,complainant's client was arrested for possessi...,2014-0067-n,nan,1,0,0.003686,0,3.360932e-11,0,0.351753
1099,complainant stated the officer constantly used...,2014-0140-c,nan,1,0,0.112272,0,7.921701e-10,0,0.301040
1193,"the complainant, who was the vietim of an atta...",2014-0273-c,nan,1,0,0.031558,0,6.532416e-05,0,0.344411
